# Electra experimenty

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from torch.utils.data import Dataset
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
from typing import Dict, List
import transformers
from transformers import ElectraModel, ElectraConfig, ElectraTokenizer, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install transformers==4.30

## Načítanie a príprava dát

In [ ]:
dataset = pd.read_csv('../Data/final_dataset_2_balanced.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['processed_text'], dataset['author_id'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.transform(y_valid)
y_test = label_encoder.transform(y_test)

## Electra

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    precision = precision_score(y_true=labels, y_pred=pred, average='macro', zero_division=0)
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
x_train = list(X_train)
x_val = list(X_valid)
x_test = list(X_test)

y_train = list(y_train)
y_val = list(y_valid)
y_test = list(y_test)

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_valid)
test_dataset = CustomDataset(test_encodings, y_test)

In [ ]:
labels_count = len(dataset['author_id'].unique())
print(labels_count)

20


## Experiment 1 - learning_rate = 5e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=5e-5,             #default
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.933675,0.125000,0.023707,0.130000,0.039593
2,No log,2.763957,0.177083,0.131647,0.226667,0.137059
3,No log,2.570821,0.270833,0.186131,0.293333,0.190139
4,No log,2.409332,0.343750,0.486120,0.391250,0.348489
5,2.652900,2.252208,0.406250,0.392919,0.438750,0.377862
6,2.652900,2.123342,0.437500,0.424643,0.486548,0.398452
7,2.652900,2.032084,0.468750,0.505920,0.521548,0.459791
8,2.652900,1.970647,0.447917,0.560833,0.496548,0.481219
9,2.652900,1.853521,0.510417,0.497778,0.562381,0.499204
10,1.796900,1.811178,0.489583,0.525278,0.538214,0.502581


TrainOutput(global_step=1620, training_loss=1.8378459883324894, metrics={'train_runtime': 183.2929, 'train_samples_per_second': 70.543, 'train_steps_per_second': 8.838, 'total_flos': 380579705118720.0, 'train_loss': 1.8378459883324894, 'epoch': 15.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 0.21313477,  1.5078125 ,  2.28125   , ...,  0.09619141,
         0.2208252 , -2.0820312 ],
       [-1.5683594 ,  0.94628906, -0.6308594 , ..., -1.2216797 ,
        -0.81884766, -1.7548828 ],
       [-1.3574219 , -0.5292969 , -0.69433594, ..., -1.2255859 ,
         0.5024414 , -0.99609375],
       ...,
       [ 2.078125  , -0.3696289 , -0.07635498, ...,  1.8378906 ,
        -0.23339844, -0.75683594],
       [ 0.24707031,  1.5185547 ,  2.2675781 , ...,  0.15185547,
         0.18395996, -2.0410156 ],
       [-2.2363281 , -1.5273438 , -1.6914062 , ..., -1.6035156 ,
        -0.6748047 ,  0.69189453]], dtype=float32), label_ids=array([ 2,  5, 16,  3,  4,  1, 19, 17, 13, 11,  1,  4,  4, 11,  9, 10, 12,
        8,  1,  3, 13,  7,  6, 19, 16,  9,  6,  0, 19,  7, 13, 14,  6,  8,
       13,  0,  3, 16, 19,  7, 17, 16,  5, 19, 11, 19, 19,  4,  4,  9,  8,
       14, 13, 10, 11, 19,  5,  5, 18, 13, 19, 10, 15, 16, 10,  8,  4,  3,
        0, 12,  3, 10,  2,  4, 1

## Experiment 4 - learning_rate = 8e-5

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    learning_rate=8e-5,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
)

model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=labels_count)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.995097,0.041667,0.004167,0.033333,0.007407
2,No log,2.987488,0.083333,0.074531,0.111667,0.066919
3,No log,2.906596,0.260417,0.199028,0.335000,0.220991
4,No log,2.721069,0.260417,0.261782,0.315833,0.231367
5,No log,2.538391,0.354167,0.397917,0.437917,0.357998
6,No log,2.383128,0.343750,0.382169,0.416250,0.352629
7,No log,2.250203,0.437500,0.418881,0.488333,0.427855
8,No log,2.122274,0.437500,0.447450,0.497500,0.439381
9,No log,2.000478,0.427083,0.418644,0.503631,0.412623
10,2.515200,1.891073,0.458333,0.430952,0.527024,0.446981


TrainOutput(global_step=648, training_loss=2.2877062573845004, metrics={'train_runtime': 128.2364, 'train_samples_per_second': 80.664, 'train_steps_per_second': 5.053, 'total_flos': 304463764094976.0, 'train_loss': 2.2877062573845004, 'epoch': 12.0})

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 1.4072266 ,  1.0048828 ,  2.0507812 , ..., -1.3261719 ,
        -0.19702148, -0.51171875],
       [-1.34375   ,  0.40893555, -1.046875  , ...,  1.4208984 ,
        -0.72802734, -1.0322266 ],
       [-0.6176758 ,  1.2871094 , -0.23486328, ...,  1.0380859 ,
         0.40014648, -0.83154297],
       ...,
       [ 2.1464844 ,  0.39208984,  1.3818359 , ..., -0.9160156 ,
        -0.42871094, -0.3383789 ],
       [ 1.484375  ,  1.0302734 ,  2.0332031 , ..., -1.2128906 ,
        -0.2434082 , -0.57714844],
       [-0.7475586 , -0.65966797, -0.40161133, ..., -0.5385742 ,
        -0.5776367 ,  0.45947266]], dtype=float32), label_ids=array([ 2,  5, 16,  3,  4,  1, 19, 17, 13, 11,  1,  4,  4, 11,  9, 10, 12,
        8,  1,  3, 13,  7,  6, 19, 16,  9,  6,  0, 19,  7, 13, 14,  6,  8,
       13,  0,  3, 16, 19,  7, 17, 16,  5, 19, 11, 19, 19,  4,  4,  9,  8,
       14, 13, 10, 11, 19,  5,  5, 18, 13, 19, 10, 15, 16, 10,  8,  4,  3,
        0, 12,  3, 10,  2,  4, 1